<a href="https://colab.research.google.com/github/giacomoaccursi/Tirocinio/blob/main/process_video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!rm -rf openpose 
!rm -rf /content/cmake-3.13.0-Linux-x86_64.tar.gz

In [ ]:
import os
from os.path import exists, join, basename, splitext

git_repo_url = 'https://github.com/CMU-Perceptual-Computing-Lab/openpose.git'
project_name = splitext(basename(git_repo_url))[0]
if not exists(project_name):
  # see: https://github.com/CMU-Perceptual-Computing-Lab/openpose/issues/949
  # install new CMake becaue of CUDA10
  !wget -q https://cmake.org/files/v3.13/cmake-3.13.0-Linux-x86_64.tar.gz
  !tar xfz cmake-3.13.0-Linux-x86_64.tar.gz --strip-components=1 -C /usr/local
  # clone openpose
  !git clone -q --depth 1 $git_repo_url
  !sed -i 's/execute_process(COMMAND git checkout master WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/execute_process(COMMAND git checkout f019d0dfe86f49d1140961f8c7dec22130c83154 WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/g' openpose/CMakeLists.txt
  # install system dependencies
  !apt-get -qq install -y libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler libgflags-dev libgoogle-glog-dev liblmdb-dev opencl-headers ocl-icd-opencl-dev libviennacl-dev
  # build openpose
  !cd openpose && rm -rf build || true && mkdir build && cd build && cmake .. && make -j`nproc`


In [ ]:
#dato il path di un video, esegue openpose e restituisce i dati nella cartella di destinazione
def process_video(video_name, video_path, destination_folder):
  !rm openpose.avi
  !cd /content/openpose && ./build/examples/openpose/openpose.bin --video {video_path} --write_json {destination_folder} --display 0  --write_video ../openpose.avi
  # convert the result into MP4
  !ffmpeg -y -loglevel info -i openpose.avi {video_name}

In [ ]:
from os import listdir
from os.path import isfile, join
#lista tutti i file dato un percorso
def list_all_files(input_path):
  files = [join(input_path, f) for f in listdir(input_path) if isfile(join(input_path, f))]
  return list(zip(files, listdir(input_path)))

def list_all_in_dir(input_path):
  files = [join(input_path, f) for f in listdir(input_path)]
  return files

In [ ]:
def destination_folder_from_video(video):
  destination_folder = "/content/video_output/" + video[0:-4] 
  return destination_folder

In [ ]:
!mkdir /content/video_output/
video_folder = "/content/drive/MyDrive/video"
videos = list_all_files(video_folder)
for video in videos: 
  process_video(video[1],video[0], destination_folder_from_video(video[1]))
  print("processed video ", video[0])


In [ ]:
!zip -r /content/video_data.zip /content/video_output/
from google.colab import files
files.download("/content/video_data.zip")